# Sentiment Classification with Naive Bayes

Accuracy: 0.82064

To run the code, the below paths need to be upated based on where the data is

In [ ]:
path_train_positive = "C:\\Users\\deniz tekalp\\Desktop\\aclImdb\\train\\pos"
path_train_negative = "C:\\Users\\deniz tekalp\\Desktop\\aclImdb\\train\\neg"
path_test_positive = "C:\\Users\\deniz tekalp\\Desktop\\aclImdb\\test\\pos"
path_test_negative = "C:\\Users\\deniz tekalp\\Desktop\\aclImdb\\test\\neg"

Reading the strings

In [ ]:
files_train_positive = readdir(path_train_positive)
files_train_negative = readdir(path_train_negative)
files_test_positive = readdir(path_test_positive)
files_test_negative = readdir(path_test_negative)

In [ ]:
xtrn_strings = []
for file in files_train_positive
    tmppath = string(path_train_positive,"\\", file)
    tmpfile = open(tmppath, "r")
    #push!(xtrn_strings, readlines(tmpfile)[1])
    x = readlines(tmpfile)[1]
    push!(xtrn_strings, x)
    close(tmpfile)
end

In [ ]:
for file in files_train_negative
    tmppath = string(path_train_negative,"\\", file)
    tmpfile = open(tmppath, "r")
    #push!(xtrn_strings, readlines(tmpfile)[1])
    x = readlines(tmpfile)[1]
    push!(xtrn_strings, x)
    close(tmpfile)
end

In [ ]:
xtst_strings = []
for file in files_test_positive
    tmppath = string(path_test_positive,"\\", file)
    tmpfile = open(tmppath, "r")
    x = readlines(tmpfile)[1]
    push!(xtst_strings, x)
    close(tmpfile)
end

In [ ]:
for file in files_test_negative
    tmppath = string(path_test_negative,"\\", file)
    tmpfile = open(tmppath, "r")
    push!(xtst_strings, readlines(tmpfile)[1])
    close(tmpfile)
end

Create a dict, clean data from punctutian, strip.() didn't work for me for some reason

In [114]:
dict = Dict()
w2i(x) = get!(dict, x, 1+length(dict))
UNK = w2i("<unk>")

1

In [115]:
xtrn = []
for sentence in xtrn_strings
    tmparr = split(strip(lowercase(sentence)))
    tmparr=strip.(tmparr, [','])
    tmparr=strip.(tmparr, ['!'])
    tmparr=strip.(tmparr, ['.'])
    tmparr=strip.(tmparr, ['?'])
    tmparr=strip.(tmparr, [':'])
    tmparr=strip.(tmparr, [';'])
    tmparr=strip.(tmparr, ['*'])
    tmparr=strip.(tmparr, ['\"'])
    tmparr=strip.(tmparr, ['\\'])
    tmparr=strip.(tmparr, ['\"'])
    tmparr=strip.(tmparr, [')'])
    tmparr=strip.(tmparr, ['-'])
    tmparr=strip.(tmparr, ['\''])
    tmparr=strip.(tmparr, ['('])
    tmparr=strip.(tmparr, [')'])
    tmparr=strip.(tmparr, ['.'])
    w2i.(tmparr)
    push!(xtrn, tmparr)
end

In [116]:
xtst = []
for sentence in xtst_strings
    tmparr = split(strip(lowercase(sentence)))
    tmparr=strip.(tmparr, [','])
    tmparr=strip.(tmparr, ['!'])
    tmparr=strip.(tmparr, ['.'])
    tmparr=strip.(tmparr, ['?'])
    tmparr=strip.(tmparr, [':'])
    tmparr=strip.(tmparr, [';'])
    tmparr=strip.(tmparr, ['*'])
    tmparr=strip.(tmparr, ['\"'])
    tmparr=strip.(tmparr, ['\\'])
    tmparr=strip.(tmparr, ['\"'])
    tmparr=strip.(tmparr, [')'])
    tmparr=strip.(tmparr, ['-'])
    tmparr=strip.(tmparr, ['\''])
    tmparr=strip.(tmparr, ['('])
    tmparr=strip.(tmparr, [')'])
    tmparr=strip.(tmparr, ['.'])
    push!(xtst, tmparr)
end

Redefine w2i to make sure new words are UNK and are not added to the dict

In [118]:
w2i(x) = get(dict, x, UNK)

w2i (generic function with 1 method)

Convert xtrn from strings to integers, create y vectors

In [120]:
for i in 1:length(xtrn)
    xtrn[i] = w2i.(xtrn[i])
    xtst[i] = w2i.(xtst[i])
end

In [127]:
ytrn = ones(25000)
ytrn[1:12500] .= 2
ytst = ones(25000)
ytst[1:12500] .= 2

12500-element view(::Array{Float64,1}, 1:12500) with eltype Float64:
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 ⋮  
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0

Define prior probs, not useful in this case because priors are equal

In [128]:
prior_prob_positive = sum(ytrn .-1)/length(ytrn)
prior_prob_negative = 1 - prior_prob_positive

0.5

Regularizing the amount of words

In [163]:
negative_counts = ones(length(dict)) .+ 4
positive_counts = ones(length(dict)) .+ 4

144340-element Array{Float64,1}:
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 ⋮  
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0

Count words that occur in negative/positive sentences & convert to probabilities

In [164]:
for i in 1:length(xtrn)
    for word in xtrn[i]
        if ytrn[i] == 1  #if sentiment is negative
            negative_counts[word] += 1
        else           
            positive_counts[word]  += 1
        end
    end
end

In [165]:
negative_word_probs = negative_counts ./ sum(negative_counts)
positive_word_probs = positive_counts ./ sum(positive_counts)

144340-element Array{Float64,1}:
 1.3584992604330027e-6 
 3.532098077125807e-6  
 0.0002624620571156561 
 0.015484174570415364  
 0.02246088337229509   
 6.520796450078413e-5  
 0.00043743676185942687
 0.010296337594673813  
 3.4234181362911666e-5 
 0.0030128796597883134 
 0.045751809792714745  
 0.0005300864114209577 
 0.0016255802150341309 
 ⋮                     
 1.3584992604330027e-6 
 1.3584992604330027e-6 
 1.3584992604330027e-6 
 1.3584992604330027e-6 
 1.3584992604330027e-6 
 1.3584992604330027e-6 
 1.3584992604330027e-6 
 1.3584992604330027e-6 
 1.3584992604330027e-6 
 1.3584992604330027e-6 
 1.3584992604330027e-6 
 1.3584992604330027e-6 

Sum by log probabilities rather than multiply due to underflow issues

In [166]:
function predict(sentence)
    negative_prob = log(prior_prob_negative)
    positive_prob = log(prior_prob_positive)
    for word in sentence
        negative_prob += log(negative_word_probs[word])
        positive_prob += log(positive_word_probs[word])
    end
    if negative_prob > positive_prob return 1
        else return 2
    end
end

predict (generic function with 1 method)

In [167]:
print("accuracy: ", count(predict.(xtst) .== ytst) / length(xtst))

accuracy: 0.82064